In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datav1/pubchem_binding.csv


In [2]:
smile = pd.read_csv("/kaggle/input/datav1/pubchem_binding.csv")
smile.head()

,UniprotID,Uniprot Name,Ligand SMILES,PubChem_CID
0,Q14457,BECN1_HUMAN,CCCC(=O)O,264
1,Q14457,BECN1_HUMAN,C,297
2,Q14457,BECN1_HUMAN,C[N+](C)(C)CCO,305
3,Q14457,BECN1_HUMAN,CC(CC(=O)O)O,441
4,Q14457,BECN1_HUMAN,[O-][As]([O-])[O-],544


In [3]:
smile = smile[['PubChem_CID', 'Ligand SMILES']]
print(smile.shape)
smile.head()

(4732, 2)


,PubChem_CID,Ligand SMILES
0,264,CCCC(=O)O
1,297,C
2,305,C[N+](C)(C)CCO
3,441,CC(CC(=O)O)O
4,544,[O-][As]([O-])[O-]


In [4]:
duplicates = smile[smile['PubChem_CID'].duplicated(keep=False)]
print(duplicates)

      PubChem_CID                                      Ligand SMILES
0             264                                          CCCC(=O)O
1             297                                                  C
2             305                                     C[N+](C)(C)CCO
3             441                                       CC(CC(=O)O)O
4             544                                 [O-][As]([O-])[O-]
...           ...                                                ...
4660    168291742  CCCn1c2nc(SSc3nc4n(CCC)c(=O)n(CCC)c(=O)c4[nH]3...
4661    168289886  CCn1c2nc(SSc3nc4n(CC)c(=O)n(CCc5ccc(OC)cc5)c(=...
4662    168292154  CCn1c2nc(SSc3nc4n(CC)c(=O)n(CC(C)C)c(=O)c4[nH]...
4663    168287129  CCn1c2nc(SSc3nc4n(CC)c(=O)n(CC5CCOCC5)c(=O)c4[...
4664    168278432  CCn1c2nc(SSc3nc4n(CC)c(=O)n(CCc5ccccc5)c(=O)c4...

[2927 rows x 2 columns]


In [5]:
smile = smile.drop_duplicates(subset='PubChem_CID', keep='first')

print(f"Dataframe setelah menghapus duplikat: {smile.shape}")

Dataframe setelah menghapus duplikat: (2935, 2)


In [6]:
smile

,PubChem_CID,Ligand SMILES
0,264,CCCC(=O)O
1,297,C
2,305,C[N+](C)(C)CCO
3,441,CC(CC(=O)O)O
4,544,[O-][As]([O-])[O-]
...,...,...
4727,156293737,Brc1ccc(cc1)C(=O)N1N=C(CC1c1ccccc1)c1ccc(NC(=O...
4728,156294005,CCOC(=O)Nc1ccc(cc1)C1=NN(C(C1)c1ccccc1)C(=O)c1...
4729,156294009,CN(C)C(=O)Nc1ccc(cc1)C1=NN(C(C1)c1ccccc1)C(=O)...
4730,156293651,Brc1ccc(cc1)C(=O)N1N=C(CC1c1ccccc1)c1ccc(NC(=O...


In [7]:
!pip install deepchem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 41.8 MB/s eta 0:00:00


In [8]:
!pip install pubchempy

  Preparing metadata (setup.py) ... - done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=d329e0a6b095f757bd6ed7b5645da65ce960a7f03c04f6ecb33de015dbba82ea
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [9]:
import deepchem as dc
import pubchempy as pcp
featurizer = dc.feat.PubChemFingerprint()

def get_pubchem_fingerprints(smiles_list):
    fingerprints = []
    for smiles in smiles_list:
        fingerprint = featurizer.featurize([smiles])
        fingerprints.append(fingerprint[0]) 
    return fingerprints

smile['pubchem_fingerprint'] = get_pubchem_fingerprints(smile['Ligand SMILES'].tolist())

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
[10:56:41] WARNING: not removing hydrogen atom without neighbors
[10:56:41] WARNING: not removing hydrogen atom without neighbors
[10:58:59] WARNING: not removing hydrogen atom without neighbors
[10:58:59] WARNING: not removing hydrogen atom without neighbors
[10:58:59] WARNING: not removing hydrogen atom without neighbors
[10:59:28] WARNING: not removing hydrogen atom without neighbors


In [10]:
smile

,PubChem_CID,Ligand SMILES,pubchem_fingerprint
0,264,CCCC(=O)O,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
1,297,C,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,305,C[N+](C)(C)CCO,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ..."
3,441,CC(CC(=O)O)O,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
4,544,[O-][As]([O-])[O-],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
4727,156293737,Brc1ccc(cc1)C(=O)N1N=C(CC1c1ccccc1)c1ccc(NC(=O...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ..."
4728,156294005,CCOC(=O)Nc1ccc(cc1)C1=NN(C(C1)c1ccccc1)C(=O)c1...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ..."
4729,156294009,CN(C)C(=O)Nc1ccc(cc1)C1=NN(C(C1)c1ccccc1)C(=O)...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ..."
4730,156293651,Brc1ccc(cc1)C(=O)N1N=C(CC1c1ccccc1)c1ccc(NC(=O...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ..."


In [11]:
smile.to_csv('/kaggle/working/fitursenyawa.csv', index=False)